# Projet 4 Bernadoy Corentin

Le but de ce projet est de créer une balise html `<a>` autour des textes de référencés dans les différents arrêtés municipaux mis à notre disposition.

## Imports

In [37]:
import os
import bs4
import regex as re

## Stratégie et étude du jeu de donnée

Pour une première approche, nous allons tenter d'utiliser un maximum les expressions régulières. En effet, il s'agit d'une approche simple et si elle réussit, permet de consitituer un premier jeu de données labélisées pour une étude par machine learning. 

Tout d'abord on établit la structure que l'on veut suivre : on va extraire les textes présents dans les balises via Beautifulsoup, recherche la présence de textes de lois dans ces textes extraits, les encadrer par une balise `<a>` puis on va sauvegarder le fichier html modifié dans un fichier séparé.

### Création des dossier de dépôt des résultats

On commence par créer les dossier de résultats dans l'arborescence actuelle

In [38]:
#Création des fichiers de résultat
root = "./data/"
os.makedirs(root + "results/", exist_ok= True) # créer un dir à côté du dir data
sub_data_dirs = next(os.walk(root + 'data/'))[1] # fait la liste des dossiers à créer
for sub_data_dir in sub_data_dirs:
    os.makedirs(root + "results/" + sub_data_dir, exist_ok= True) #réplique l'arborescence du dossier data

### Fonctions utiles

Il nous faut trois fonctions pour établir la structure : une première fonction `find_regex` pour évaluer les matchs avec les regex et déterminer leur position dans le texte, une fonction `rewritting` pour réécrire le texte avec les balises et enfin une fonction  `pipeline` qui itère les deux premières sur chaque texte de chaque fichier et qui sauvegarde les résultats

On écrit d'abord la list des regex que l'on va utiliser pour détecter les références à des textes de lois (plus de détails sur comment établir les regexs dans la partie suivante)

In [39]:
regex_list = [
    # regex pour les directives
    r"directive[\sA-Za-zÀ-ÿ]+\d{4}/\d+/[A-Z]{2}",
    # regex pour les décrets
    r"[Dd]écret(?:\s+n°|\s+no)?\s*[./\d\-]+(?:\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|[Dd]écret\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}",
    # regex pour les articles du code de l'environnemnent
    r"article(.+?)[Cc]ode\s+de\s+l[’']environnement|[\sA-Za-zÀ-ÿ]{2}\s+[Cc]ode\s+de\s+l[’']environnement",
    # regex pour les lois
    r"\s[Ll]oi(?:\s+n°|\s+no)?\s*[\d\-\s]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?",
    # regex pour les circulaires
    r"circulaire[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)(?:[A-Z/\d]+)\s+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|circulaire[\sA-Za-zÀ-ÿ]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})",
    # regex les arrêtés pour 
    r"[Aa]rrêté[\sA-Aa-zÀ-ÿ]+(?:\s+n°|\s+no)\s*[\d\-]+|[Aa]rrêté[\sA-Aa-zÀ-ÿ]+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}"
]

In [40]:
cde = regex_list[-1]

On écrit les fonctions

In [41]:
def find_regex(regex_list, text):
    occurences = [] #On crée une liste pour récupérer les positions où insérer les balises
    for regex in regex_list:
        for occurence in re.finditer(regex, text):
            occurences.append((occurence.start(0),occurence.end(0)))# On récupère les positions où insérer les balises
    return((len(occurences) > 0), occurences)

In [42]:
### only for tests (if somebody is curious enough to look until here in the commits :))

with open('data/data/0022200009/1999-12-21_AP-auto_pixtral.html', 'r') as f:
    soup = bs4.BeautifulSoup(f, 'html')   
    whole_text = soup.find_all(['div', 'td', 'h2'])
    i = 0
    for tag in whole_text:
        for occurence in re.finditer(cde, tag.text):
            i += 1
            print((occurence.start(0),occurence.end(0)))
            print(tag.text[occurence.start(0):occurence.end(0)])
            print(tag.text)
    
print(i)

(10, 35)
arrêté du 25 janvier 1991

    VU l'arrêté du 25 janvier 1991 relatif aux installations d'incinérations de résidus ;
   
(41, 69)
arrêté préfectoral n° 99-930

    VU l'enquête publique prescrite par arrêté préfectoral n° 99-930 du 5 mai 1999 qui s'est déroulée du 7 juin au 7 juillet 1999 inclus ;
   
(10, 39)
arrêté préfectoral n° 99-2694

    VU l'arrêté préfectoral n° 99-2694 du 10 novembre 1999 portant prorogation du délai d'instruction du dossier ;
   
(138, 160)
arrêté du 23 août 1989

     4.4.1- Définitions:
     Sont considérés comme déchets d'activités de soins et, à ce titre, admis à l'incinération conformément à l'arrêté du 23 août 1989, les déchets définis à l'article R 44-1 du code de la santé publique (décret n° 97-1049 du 6 novembre 1997), l'ensemble de ces déchets étant admis à raison de 10 % en mélange avec les ordures ménagères au niveau des fours d'incinération.
    
(177, 201)
arrêté du 4 janvier 1985

     Tout contenu de déchets d'activités de soins arri

In [43]:
def rewriting(soup, tag, occurences): # fonction de réécriture
    sorted_occurences = sorted(occurences) # on trie la liste pour insérer les balises une par une
    spliced_text = [] # on créé une liste qui permettra de découper le texte
    start = 0 # on indique le premier indice à sélectionner
    for occ in sorted_occurences:
        spliced_text.append(tag.text[start:occ[0]]) #on découpe le texte jusqu'à rencontrer une balise à insérer
        spliced_text.append(tag.text[occ[0]:occ[1]]) #on découpe le texte à baliser
        start = occ[1] #on repart de après la balise
    spliced_text.append(tag.text[start:]) # on ajoute le segment de la dernière balise à la fin
    tag.string = '' #on efface le texte présent pour le baliser avec le texte découper
    for i, substring in enumerate(spliced_text):
        if (i%2 == 0): # si le segment a un indice pair, il est hors de balises : on l'ajoute sans le modifier
            tag.append(substring)
        if (i%2 != 0):  #sinon : on l'ajoute en le balisant : il faut créer un nouveau tag 'a'
            a_tag = soup.new_tag('a')
            a_tag.string = substring
            tag.append(a_tag)

In [44]:
root = "./data/"
def pipeline(root = root): # on créer la fonction pour itérer sur tous les fichiers
    data_dirs = root + 'data/'  #dossier des données
    sub_data_dirs = next(os.walk(data_dirs))[1] #sous-dossiers des données
    res_dirs = root + 'results/' # dossier des résultats
    for sub_data_dir in sub_data_dirs: # pour chaque sous dossier
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2] #on récupère les noms des fichiers
        for file_name in file_names: # pour chaque fichier
            data_path = data_dirs + sub_data_dir + '/' + file_name #on récupère le path
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html' #on récupère le path où copier les résultat
            with open(data_path, 'r') as f: # pour chaque fichier de donnée
                soup = bs4.BeautifulSoup(f, 'html')   # on transforme en beautiful soup
                whole_text = soup.find_all(['div', 'td', 'h2']) # on extrait le texte des balises où il peut être
                for tag in whole_text: #pour chaque tag de texte
                    if (not(tag.find('h1'))): # on exclue le div qui contient le titre, car ce dernier contient toujours le nom du texte présenté dans le document, ce qui n'est pas une référence externe
                        found, occurences = find_regex(regex_list, tag.text) # on cherche des regex
                        if found: # si on en trouve
                            rewriting(soup, tag, occurences) #on réécrit le texte

            with open(res_data_path, 'w', encoding= "utf-8") as f:#on ouvre et on copie les résultats
                    f.write(str(soup))

In [45]:
pipeline() # cellule d'exécution de pipeline

./data/data/0003013459/2020-04-20_AP-auto_initial_pixtral.html
./data/data/0003013459/2021-09-24_AP-auto_refonte_pixtral.html
./data/data/0003013459/2023-02-23_APC-auto_pixtral.html
./data/data/0005205103/2002-12-11_AP-auto_initial_pixtral.html
./data/data/0005205103/2004-04-16_AP-auto_refonte_pixtral.html
./data/data/0005205103/2004-05-04_APC-auto_pixtral.html
./data/data/0005205103/2004-05-04_APC-TAR_pixtral.html
./data/data/0005205103/2006-01-31_AP-audit_pixtral.html
./data/data/0005205103/2007-10-25_AP-auto_refonte_pixtral.html
./data/data/0005205103/2008-08-14_AP-radioactif_pixtral.html
./data/data/0005205103/2008-11-26_AP-audit_pixtral.html
./data/data/0005205103/2009-09-11_AP-auto_refonte_pixtral.html
./data/data/0005205103/2010-04-02_AP-MU_pixtral.html
./data/data/0005205103/2011-07-18_AP-RSDE_pixtral.html
./data/data/0005205103/2012-01-11_AP-auto_refonte_pixtral.html
./data/data/0005205103/2012-01-11_AP-auto_refonte_pixtral_old.html
./data/data/0005205103/2013-10-23_AP-auto_re

## Comment établir les regexs :

On écrit une fonction `get_example` qui permet de récupérer toutes les balises contenant un mot de texte de loi à référencer, ce qui nous permet de récupérer toutes les occurences d'un type de loi dans les documents et donc d'avoir une idée de quelle regexp utiliser. On sauvegarde ces textes dans un fichier text séparé afin de pouvoir l'étudier plus facilement.

In [46]:
#### ne pas supprimer : permet de savoir quel format peut avoir chaque texte de loi
def get_examples(Loi, name):
    root = "./data/"
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    examples = [] # on récupère le texte des exemples dans une liste
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            with open(data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if(find_regex(Loi, tag.text)[0]):
                        examples.append(tag.text)

    with open(f"example_{name}.txt", 'w') as f: # on envoie les résultats dans un fichier texte
        for example in examples:
            f.write(f"{example}\n")

In [47]:
get_examples([r"[Aa]rrêté"], "Arrete") # exemple pour les arrêtés

À l'aide des documents d'exemple, on peut déterminer un pattern chaque texte de loi, qui permet de l'identifier au mieux:
- Pour les directives, la structure est généralement la suivante : le mot 'directive' suivi optionnellement du mot 'européenne' et d'un code d'identification sous le format '####/##/AA' avec # des chiffres et A des lettres. À noter qu'il peut y avoir plus que deux chiffres au centre du code. On décide donc comme regex une expression qui encapsule du mot directive à la fin du code, ce qui se traduit ici par : `r"directive[\sA-Za-zÀ-ÿ]+\d{4}/\d+/[A-Z]{2}"`.
- Pour les décrets, ils sont généralement de la forme suivante : ils commencent par le mot 'décret', avec parfois un D majuscule, puis il peut y avoir soit un numéro indiqué (écrit no ou n°) et contenant des chiffres, des tirets, des points ou des /, puis possède une date de la forme (JJ mois AAAA) avec J et A des chiffres, mois le mois en toute lettre. L'autre forme possible est avec la date directement après le mot décret, de la même forme que pour la première possibilité. Cela donne en regex : `r"[Dd]écret(?:\s+n°|\s+no)?\s*[\d\-/.]+(?:\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|[Dd]écret\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}"`
- Pour le code de l'environnement, on peut tenter de récupérer des références aux articles pour pouvoir extraire des références plus précises, dans ce cas, le pattern est : "article" + d'éventuels autres articles et des lettres et chiffres faisant référence à l'article, puis le code de l'environnement avec éventuellement un C majusule. Attention, deux types d'apostrophe sont présents. S'il n'y a pas d'article, on tente d'identifier directement la référence au code de l'environnemnet, ce qui donne en tout : `r"article(.+?)[Cc]ode\s+de\s+l[’']environnement|[\sA-Za-zÀ-ÿ]{2}\s+[Cc]ode\s+de\s+l[’']environnement"`
- Pour les lois, la logique est similaire que pour les décrets, avec optionnellement un numéro de loi ou une date écrite.  En regex : `r"\s[Ll]oi(?:\s+n°|\s+no)?\s*[\d\-\s]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?"`
- Pour les circulaires, la logique est similaire au décret, avec quelques différences de format des numéros. On essaye également de capter d'éventuels mots entre les deux, qui qualifient la circulaire. En regex : `r"circulaire[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)(?:[A-Z/\d]+)\s+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|circulaire[\sA-Za-zÀ-ÿ]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})"`
- Pour les arrêtés, la logique est identique à celle des circulaires, ce qui donne en regex : `r"[Aa]rrêté[\sA-Aa-zÀ-ÿ]+(?:\s+n°|\s+no)\s*[\d\-]+|[Aa]rrêté[\sA-Aa-zÀ-ÿ]+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}"`

## Comparaisons des fichiers de sorties pour analyse des résultats.

Afin d'avoir une idée plus générale que d'éplucher les fichiers, on peut comparer les fichiers de résultat et de data pour évaluer la précision de la méthode par regex

In [48]:
def compare(Loi):
    root = "./data/"
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    original = [] # on récupère le texte des exemples dans une liste
    results = []
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            with open(data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if(find_regex(Loi, tag.text)[0]):
                        original.append(tag.text)

            with open(res_data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if(find_regex(Loi, tag.text)[0]):
                        results.append(tag.text)
    return(original, results)
                

In [50]:
original, result = compare(r"code de l'environnement")